Epipolar Geometry implementation from https://docs.opencv.org/3.2.0/da/de9/tutorial_py_epipolar_geometry.html.

Our hope is to see if we can just use together superpixels that are overlapping with epipolars.

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
img1 = cv2.imread('../data/myleft.jpg',0)  #queryimage # left image
img2 = cv2.imread('../data/myright.jpg',0) #trainimage # right image
sift = cv2.SIFT()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)
good = []
pts1 = []
pts2 = []
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

AttributeError: module 'cv2' has no attribute 'SIFT'